# 变量不是盒子，而是标注

In [1]:

a = [1,2,3]
b = a
a.append(4)
b

[1, 2, 3, 4]

In [2]:
a = 4
b = a
a = 6
b

4

# 标识、相等性和别名

In [4]:
liu = {'name':'zhongkai liu', 'born':1990}
xxx = liu
xxx is liu

True

In [5]:
id(xxx), id(liu)

(140294907731584, 140294907731584)

In [6]:
xxx['balance'] = 950
liu

{'name': 'zhongkai liu', 'born': 1990, 'balance': 950}

- == 运算符比较两个对象的值（对象中保存的数据）
- is 比较对象的标识（计算速度快）

`a == b`其背后计算为：

`a.__eq__(b)`

In [7]:
alex = {'name': 'zhongkai liu', 'born': 1990, 'balance': 950}

alex == liu

True

In [8]:
alex is liu

False

is 运算符比 == 速度快，因为它不能重载，所以 Python 不用寻找并调
用特殊方法，而是直接比较两个整数 ID。而 a == b 是语法糖，等同
于 a.\_\_eq\_\_(b)。继承自 object 的 \_\_eq\_\_ 方法比较两个对象的
ID，结果与 is 一样。但是多数内置类型使用更有意义的方式覆盖了
\_\_eq\_\_ 方法，会考虑对象属性的值。相等性测试可能涉及大量处理工 作，例如，比较大型集合或嵌套层级深的结构时。

# 元组的相对不可变性

In [9]:
t1 = (1,2,[30,40])
t2 = (1,2,[30,40])

t1 == t2

True

In [10]:
t1 is t2

False

In [11]:
id(t1[-1])

140294905825984

In [12]:
t1[-1].append(99)

id(t1[-1])

140294905825984

In [13]:
t1

(1, 2, [30, 40, 99])

In [14]:
t1 == t2

False

# 默认做浅复制

为数组制作一个副本的方法（浅复制）
- 构造器
- \[:\]法

In [15]:
# 创建副本（可散列数据）
l1 = [3,[55,44],(7,8,9)]

# 这样只是将变量作为数据的标注进行了绑定，并没有创建副本！
l2 = l1
l2 is l1

True

In [16]:
# 使用构造法创建副本
l3 = list(l1)
l3 is l1

False

In [17]:
l3 == l1

True

In [22]:
# 使用[:]创建副本
l4 = l1[:]
l4 is l1

False

In [21]:
l4 == l1

True

# 浅复制和深复制

- copy.copy():浅复制
- copy.deepcopy():深复制

In [24]:
# 测试浅复制和深复制
class Bus:

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)


In [28]:
import copy

bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1) # 浅复制
bus3 = copy.deepcopy(bus1) # 深复制

id(bus1),id(bus2),id(bus3)

(140294918052208, 140294918050000, 140294918053360)

In [29]:
bus1.drop('Bill')
# 浅复制结果
bus2.passengers

['Alice', 'Claire', 'David']

In [32]:
# 深复制不会有这样的问题
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

# 深复制甚至可以应付循环引用

In [33]:
# 循环引用
a = [10, 20]
b = [a, 30]
a.append(b)
a

[10, 20, [[...], 30]]

In [35]:
# 深复制依然可以应付这样的循环引用
c = copy.deepcopy(a)
c

[10, 20, [[...], 30]]

# 函数的参数作为引用时

Python 唯一支持的参数传递模式是共享传参(call by sharing)。

共享传参指函数的各个形式参数获得实参中各个引用的副本。也就是 说，函数内部的形参是实参的别名。

**因此函数可能会修改接收到的任何可变对象**

In [39]:
def f(a, b):
    a += b
    return a

# 不可变类型
t = (10, 20)
u = (30, 40)
f(t, u), t ,u

((10, 20, 30, 40), (10, 20), (30, 40))

In [40]:
# 可变类型
a = [1, 2]
b = [3, 4]
f(a, b), a, b

([1, 2, 3, 4], [1, 2, 3, 4], [3, 4])

## 不要使用可变类型作为参数的默认值

In [42]:
# 如果bus类的passenger默认值不是None而是一个列表，这样的“聪明举动”会导致诡异结果：

class HauntedBus:
    """备受幽灵乘客折麽的校车"""

    def __init__(self, passengers=[]) :
        '''直接使用可变类型作为默认值'''
        self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)


In [43]:
hbus1 = HauntedBus(['Alice', 'Bill'])
hbus1.passengers

['Alice', 'Bill']

In [44]:
hbus1.pick('Charlie')
hbus1.drop('Alice')
hbus1.passengers

['Bill', 'Charlie']

In [45]:
# 创建实例时，使用默认值
hbus2 = HauntedBus()
hbus2.pick('Carrie')
hbus2.passengers

['Carrie']

In [46]:
# bus3也使用默认值
hbus3 = HauntedBus()
hbus3.passengers

['Carrie']

In [47]:
# 虽然是不同的实例，但是因为使用了默认值，他们的属性却指向同一个位置，即默认列表
# 没有指定初始乘客的 HauntedBus 实例会共享同一个乘客列表。
hbus3.passengers is hbus2.passengers

True

出现这个问题的根源是，默认值在定义函数时计算 (通常在加载模块时)，因此默认值变成了函数对象的属性。因此，如 果默认值是可变对象，而且修改了它的值，那么后续的函数调用都会受 到影响。

In [48]:
# 此时HauntedBus类的属性也已经变了：
HauntedBus.__init__.__defaults__

(['Carrie'],)

In [51]:
# 实例的属性不过是类属性的别称，他们指向了同一块内存。
HauntedBus.__init__.__defaults__[0] is hbus3.passengers

True

# 防御可变参数

第一个例子中，我们将传入的人员列表进行了浅复制，因此我们对乘客的修改并不会原始形参的列表，当我们不实用这种方法时，就会出现一些“惊讶”结果：


In [54]:
class TwilightBus:

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
             # 这里只是将别称绑定到形参上，并没有创建新的列表。
            # self.passengers = list(passengers)
            self.passengers = passengers
    
    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [55]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
tbus = TwilightBus(basketball_team)
tbus.drop('Tina')
tbus.drop('Maya')
# 实例的操作修改到了传入的形参，有时这并是不我们想要的。
basketball_team

['Sue', 'Diana', 'Pat']

### 除非这个方法确实想修改通过参数传入的对象，否则在类中 直接把参数赋值给实例变量之前一定要三思，因为这样会为参数对 象创建别名。如果不确定，那就创建副本。这样客户会少些麻烦。

# del 和 垃圾回收
del 语句删除名称，而不是对象。del 命令可能会导致对象被当作垃圾
回收，但是仅当删除的变量保存的是对象的最后一个引用，或者无法得 到对象时。2 重新绑定也可能会导致对象的引用数量归零，导致对象被 销毁。


In [65]:
# 观察是如何GC的：
import weakref

# 创建一个集合
s1 = {1,2,3}
s2 = s1 # s2是s1的别称
def bye():
    print('Gone with the wind ...')

# finalize()函数可以在第一个参数被GC时回调第二参数（传入一个函数）
ender = weakref.finalize(s1, bye)
ender.alive

True

In [66]:
# s1绑定的集合并没有直接删除，而是删除了这个别名
# 此时集合还绑定给了s2，因此集合还没有回收
del s1
ender.alive

True

In [67]:
# 把s2绑定给了其他数据后,集合被回收了，并且回调了我们的bye()函数：
s2 = 'spam'

Gone with the wind ...


In [68]:
ender.alive

False

In [ ]:
# 引用分为两种：
# 强引用 ：会影响GC
# 弱引用 ：不会影响GC

# 弱引用

弱引用不会增加对象的引用数量。引用的目标对象称为所指对象 (referent)。因此我们说，弱引用不会妨碍所指对象被当作垃圾回收。

In [113]:
# 控制台会话中，会自动使用_变量来绑定到不为None的结果上！
import weakref
a_set = {0, 1 , 2}
wref = weakref.ref(a_set) # 创建弱引用
wref

<weakref at 0x7f98f35fc630; to 'set' at 0x7f98f35b9740>

In [114]:
wref()

{0, 1, 2}

In [115]:
a_set = {3,4,5} # 不再指向{0,1}

In [116]:
wref()

{0, 1, 2}

In [117]:
wref

<weakref at 0x7f98f35fc630; to 'set' at 0x7f98f35b9740>

In [120]:
wref() is None

False

In [122]:
print('nothing')

nothing


In [123]:
wref() is None

False

In [125]:
8000.0/12900.0

0.6201550387596899

# WeakValueDictionary
WeakValueDictionary 类实现的是一种可变映射，里面的值是对象 的弱引用。被引用的对象在程序中的其他地方被当作垃圾回收后，对应 的键会自动从 WeakValueDictionary 中删除。因此，WeakValueDictionary 经常用于缓存。

In [12]:
import weakref

class Cheese:
    def __init__(self, kind):
        self.kind = kind
    
    def __repr__(self):
        return 'Cheese(%r)' % self.kind

    
stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'),
            Cheese('Brie'), Cheese('Parmesan')]

for cheese in catalog:
    stock[cheese.kind] = cheese

sorted(stock.keys())

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']

In [13]:
# 由于cheese作为全局变量，其实还没有del，
# 它还在引用catalog列表中最后一个元素
del catalog
sorted(stock.keys())

['Parmesan']

In [14]:
del cheese
sorted(stock.keys())

[]

## WeakKeyDictionary
可以为应用中其他部分拥有的**对象附加数据**，这样就无需为对象添加属性。这对覆盖属性访问权限的对象尤其有用。

## WeakSet
“保存元素弱引用的集合类。元素没有强引用时，集合会把它 删除。”

如果一个类需要知道所有实例，一种好的方案是创建一个WeakSet 类型的类属性，保存实例的引用。

# 弱引用的局限

基本 的 list 和 dict 实例不能作为弱引用所指对象，但是它们的子类可以轻松地 解决这个问题。

In [16]:
class Mylist(list):
    """list的子类，实例可以作为弱引用的目标"""


a_list = Mylist(range(10))
# a_list可以作为弱引用的目标
wref_to_a_list = weakref.ref(a_list)

千万不要依赖字符串或整数的驻留!比较字符串或整数是否 相等时，应该使用 ==，而不是 is。驻留是 Python 解释器内部使用 的一个特性。